# Accessibility measures: distance to public transport terminals

**Vilém Knap**

This notebook derives proximity-based public transport accessibility measures for each residential land price polygon.  
The output is a single enriched GeoDataFrame containing distance features (in meters) to the nearest public transport stop by mode (metro, tram, bus, rail).  
These features serve as model inputs for subsequent exploratory analysis and spatial regression.


## 1. Load preprocessed spatial layers

All inputs in this notebook are taken from `data/processed/`, produced in `01_Data_Preprocessing.ipynb`.  
They share a common CRS (`EPSG:5514`) and are already restricted to the territory of Prague.


In [45]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd

TARGET_CRS = "EPSG:5514"

DATA_DIR = "data/processed"

PRICE_RES_PATH = os.path.join(DATA_DIR, "price_residential_prague.gpkg")  # adjust name if needed
STOPS_TRAM_PATH  = os.path.join(DATA_DIR, "stops_tram.gpkg")
STOPS_METRO_PATH = os.path.join(DATA_DIR, "stops_metro.gpkg")
STOPS_BUS_PATH   = os.path.join(DATA_DIR, "stops_bus.gpkg")
STOPS_RAIL_PATH  = os.path.join(DATA_DIR, "stops_rail.gpkg")

ROUTES_TRAM_PATH  = os.path.join(DATA_DIR, "routes_tram.gpkg")
ROUTES_METRO_PATH = os.path.join(DATA_DIR, "routes_metro.gpkg")
ROUTES_BUS_PATH   = os.path.join(DATA_DIR, "routes_bus.gpkg")
ROUTES_RAIL_PATH  = os.path.join(DATA_DIR, "routes_rail.gpkg")


price = gpd.read_file(PRICE_RES_PATH)
stops_tram  = gpd.read_file(STOPS_TRAM_PATH)
stops_metro = gpd.read_file(STOPS_METRO_PATH)
stops_bus   = gpd.read_file(STOPS_BUS_PATH)
stops_rail  = gpd.read_file(STOPS_RAIL_PATH)

routes_tram  = gpd.read_file(ROUTES_TRAM_PATH)
routes_metro = gpd.read_file(ROUTES_METRO_PATH)
routes_bus   = gpd.read_file(ROUTES_BUS_PATH)
routes_rail  = gpd.read_file(ROUTES_RAIL_PATH)


# quick CRS sanity check
for name, gdf in {
    "price": price,
    "stops_tram": stops_tram,
    "stops_metro": stops_metro,
    "stops_bus": stops_bus,
    "stops_rail": stops_rail,
    "routes_tram": routes_tram,
    "routes_metro": routes_metro,
    "routes_bus": routes_bus,
    "routes_rail": routes_rail  

}.items():
    print(name, "rows:", len(gdf), "| CRS:", gdf.crs)


price rows: 5728 | CRS: EPSG:5514
stops_tram rows: 594 | CRS: EPSG:5514
stops_metro rows: 120 | CRS: EPSG:5514
stops_bus rows: 2820 | CRS: EPSG:5514
stops_rail rows: 47 | CRS: EPSG:5514
routes_tram rows: 35 | CRS: EPSG:5514
routes_metro rows: 3 | CRS: EPSG:5514
routes_bus rows: 286 | CRS: EPSG:5514
routes_rail rows: 31 | CRS: EPSG:5514


### Sanity check: stop layers by mode

Before computing distances, we verify that each mode-specific stop layer:
- contains features (non-empty),
- has point geometry,
- and uses the same projected CRS as the residential polygons (EPSG:5514).

If any layer is empty, distance features for that mode cannot be computed.

In [35]:
def qc_gdf(gdf, name):
    print(f"\n{name}")
    print("  rows:", len(gdf))
    print("  CRS:", gdf.crs)
    if len(gdf) > 0:
        print("  geom types:", gdf.geom_type.value_counts().to_dict())
        print("  columns head:", list(gdf.columns)[:15])

qc_gdf(stops_tram,  "stops_tram")
qc_gdf(stops_metro, "stops_metro")
qc_gdf(stops_bus,   "stops_bus")
qc_gdf(stops_rail,  "stops_rail")



stops_tram
  rows: 594
  CRS: EPSG:5514
  geom types: {'Point': 594}
  columns head: ['stop_id', 'stop_name', 'wheelchair', 'platform_c', 'asw_node_i', 'asw_stop_i', 'routes_id', 'routes_nam', 'is_night', 'is_regiona', 'route_type', 'zones_id', 'on_request', 'validity', 'geometry']

stops_metro
  rows: 120
  CRS: EPSG:5514
  geom types: {'Point': 120}
  columns head: ['stop_id', 'stop_name', 'wheelchair', 'platform_c', 'asw_node_i', 'asw_stop_i', 'routes_id', 'routes_nam', 'is_night', 'is_regiona', 'route_type', 'zones_id', 'on_request', 'validity', 'geometry']

stops_bus
  rows: 2820
  CRS: EPSG:5514
  geom types: {'Point': 2820}
  columns head: ['stop_id', 'stop_name', 'wheelchair', 'platform_c', 'asw_node_i', 'asw_stop_i', 'routes_id', 'routes_nam', 'is_night', 'is_regiona', 'route_type', 'zones_id', 'on_request', 'validity', 'geometry']

stops_rail
  rows: 47
  CRS: EPSG:5514
  geom types: {'Point': 47}
  columns head: ['stop_id', 'stop_name', 'wheelchair', 'platform_c', 'asw_node

### 1.1 Identify stop ID column

For diagnostics and reproducibility, we store the ID of the nearest stop
(for each mode). This requires a valid stop identifier column.

In [36]:
print("STOP_ID_COL:", STOP_ID_COL)

for name, gdf in [("tram", stops_tram), ("metro", stops_metro), ("bus", stops_bus), ("rail", stops_rail)]:
    has = STOP_ID_COL in gdf.columns
    print(f"{name}: STOP_ID_COL present? {has}")
    if has:
        print("  example IDs:", gdf[STOP_ID_COL].head(5).tolist())


STOP_ID_COL: stop_id
tram: STOP_ID_COL present? True
  example IDs: ['U1006Z1', 'U1006Z2', 'U100Z1', 'U100Z2', 'U100Z3']
metro: STOP_ID_COL present? True
  example IDs: ['U1000Z101', 'U1000Z102', 'U1007Z101', 'U1007Z102', 'U100Z101']
bus: STOP_ID_COL present? True
  example IDs: ['U1000Z1', 'U1000Z10', 'U1000Z11', 'U1000Z12', 'U1000Z15']
rail: STOP_ID_COL present? True
  example IDs: ['U100Z301', 'U1048Z301', 'U1051Z301', 'U1069Z301', 'U1085Z301']


## 2. Accessibility metric choice: Euclidean distance

Public transport accessibility can be measured using detailed network-based travel times or geometric proximity measures.  
In this project, accessibility is approximated using Euclidean distance (straight-line distance) from each residential polygon to the nearest public transport stop of a given mode.

This choice is intentional:

- It mirrors common real-estate communication (e.g., “5 minutes to metro”), where accessibility is often expressed as approximate proximity rather than exact route-based travel time.
- It provides a transparent, assumption-light proxy suitable for city-wide exploratory modelling.
- It allows comparability across modes using a consistent definition.

Euclidean distance is therefore interpreted as a **relative accessibility proxy**, not as exact walking distance.


## 3. Compute accessibility distances using 1-nearest neighbour (KNN, k=1)

In this notebook, we enrich the residential land price polygons with accessibility features:
for each polygon, we compute the Euclidean distance to the **nearest public transport stop** for each mode (tram, metro, bus, rail).

We implement this as a **1-nearest neighbour (k=1)** spatial join using GeoPandas `sjoin_nearest`.
This approach is transparent and efficient, and (unlike a unary-union shortcut) it can be extended later
to keep identifiers of the nearest stops for diagnostic purposes.

In [57]:
def add_nearest_stop(
    polygons_gdf,
    centroids_gdf,
    stops_gdf,
    mode_name,
    stop_id_col=STOP_ID_COL
):
    """
    Adds nearest stop ID and centroid-to-stop distance (meters)
    for a given transport mode.

    Important: If multiple stops are exactly equally nearest (ties),
    GeoPandas can return multiple rows per centroid. We enforce 1 row per centroid.
    """

    if stop_id_col not in stops_gdf.columns:
        raise ValueError(
            f"Stop layer for '{mode_name}' does not contain ID column '{stop_id_col}'. "
            f"Available columns: {list(stops_gdf.columns)}"
        )

    stops_slim = stops_gdf[[stop_id_col, "geometry"]].copy()

    dist_col = f"dist_{mode_name}_m"
    id_col_out = f"nearest_{mode_name}_id"

    nearest = gpd.sjoin_nearest(
        centroids_gdf,
        stops_slim,
        how="left",
        distance_col=dist_col
    ).rename(columns={stop_id_col: id_col_out})

    if "index_right" in nearest.columns:
        nearest = nearest.drop(columns="index_right")

    # --- Enforce exactly one match per centroid (handle ties) ---
    # Sort by distance, then keep the first row per left index (centroid index)
    nearest = nearest.sort_values(by=[dist_col, id_col_out])
    nearest = nearest[~nearest.index.duplicated(keep="first")]
    # -----------------------------------------------------------

    # Align to polygon index (safe even if something reorders)
    polygons_gdf[id_col_out] = nearest.reindex(polygons_gdf.index)[id_col_out]
    polygons_gdf[dist_col] = nearest.reindex(polygons_gdf.index)[dist_col]

    return polygons_gdf

out = price_m.copy()
out = add_nearest_stop(out, centroids, stops_tram_m,  "tram")
out = add_nearest_stop(out, centroids, stops_metro_m, "metro")
out = add_nearest_stop(out, centroids, stops_bus_m,   "bus")   
out = add_nearest_stop(out, centroids, stops_rail_m,  "rail")

## Export dataset

In [61]:
import os

# Output directory (already defined earlier)
DATA_DIR = "data/processed"

# Output path
OUT_GPKG_PATH = os.path.join(
    DATA_DIR,
    "price_residential_prague_nearest_stops_by_mode.gpkg"
)

# Save GeoPackage
out.to_file(
    OUT_GPKG_PATH,
    driver="GPKG"
)

print("Saved to:", OUT_GPKG_PATH)


Saved to: data/processed\price_residential_prague_nearest_stops_by_mode.gpkg
